# Making a Conversational Form

### Setting up Conversation Filtering



In [2]:
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
import os
from typing import Optional
from pydantic import BaseModel, Field

load_dotenv()

llm = AzureChatOpenAI(
    azure_endpoint=os.getenv("AISTUDIO_AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AI_STUDIO_AZURE_OPENAI_GPT4o_DEPLOYMENT"),
    openai_api_version="2024-02-01",
    api_key=os.getenv("AISTUDIO_AZURE_OPENAI_KEY")
)

In [41]:
class PersonalDetails(BaseModel):
    first_name: str = Field(
        ...,
        description="This is the first name of the user.",
    )
    last_name: str = Field(
        ...,
        description="This is the last name or surname of the user.",
    )
    full_name: str = Field(
        ...,
        description="Is the full name of the user ",
    )
    phone_nr: str = Field(
        ...,
        description="Is the phone number of the user ",
    )
    city: str = Field(
        ...,
        description="The name of the city where someone lives",
    )
    email: str = Field(
        ...,
        description="an email address that the person associates as theirs",
    )
    language: str = Field(
        ..., enum=["spanish", "english", "french", "german", "italian"]
    )
structured_llm = llm.with_structured_output(PersonalDetails)


In [42]:
structured_llm.invoke("Hi my name is David Jones and I live in Melbourne Australia.")

PersonalDetails(first_name='David', last_name='Jones', full_name='David Jones', phone_nr='', city='Melbourne', email='', language='english')

In [43]:
res = structured_llm.invoke("Hi my name is Chatree Kongsuwan  and I live in Bangkok. you can contact me at chatree@gmail.com")
print(res)
print(res.email)

first_name='Chatree' last_name='Kongsuwan' full_name='Chatree Kongsuwan' phone_nr='' city='Bangkok' email='chatree@gmail.com' language='english'
chatree@gmail.com


## Doing the full thing in a natural conversation

In [45]:
user_123_personal_details = PersonalDetails(first_name="",
                                last_name="",
                                full_name="",
                                phone_nr="",
                                city="",
                                email="",
                                language="")

In [46]:
user_123_personal_details

PersonalDetails(first_name='', last_name='', full_name='', phone_nr='', city='', email='', language='')

In [47]:
def check_what_is_empty(user_personal_details):
    ask_for = []
    # Check if fields are empty
    dictionary = user_personal_details.model_dump(exclude_unset=True)
    # check if the field is empty
    for field, value in dictionary.items():
        if value in [None, "", 0]:  # You can add other 'empty' conditions as per your requirements
            print(f"Field '{field}' is empty.")
            ask_for.append(f'{field}')
    return ask_for

In [48]:
ask_for = check_what_is_empty(user_123_personal_details)
ask_for

Field 'first_name' is empty.
Field 'last_name' is empty.
Field 'full_name' is empty.
Field 'phone_nr' is empty.
Field 'city' is empty.
Field 'email' is empty.
Field 'language' is empty.


['first_name',
 'last_name',
 'full_name',
 'phone_nr',
 'city',
 'email',
 'language']

In [49]:
## checking the response and adding it
def add_non_empty_details(current_details: PersonalDetails, new_details: PersonalDetails):
    new_dict = new_details.model_dump(exclude_unset=True)
    non_empty_details = {k: v for k, v in new_dict.items() if v not in [None, ""]}
    updated_details = current_details.model_copy(update=non_empty_details)
    return updated_details

In [50]:
user_123_personal_details = add_non_empty_details(user_123_personal_details,res)
user_123_personal_details

PersonalDetails(first_name='Chatree', last_name='Kongsuwan', full_name='Chatree Kongsuwan', phone_nr='', city='Bangkok', email='chatree@gmail.com', language='english')

In [53]:
#adding more information based on the response
res = structured_llm.invoke("My phone number is 123456789")
print(res)
print(res.email)

first_name='' last_name='' full_name='' phone_nr='123456789' city='' email='' language='english'



In [52]:
user_123_personal_details = add_non_empty_details(user_123_personal_details,res)
user_123_personal_details

PersonalDetails(first_name='John', last_name='Doe', full_name='John Doe', phone_nr='123456789', city='New York', email='john.doe@example.com', language='english')

## Putting it together with a LLMChain as well


In [54]:
from langchain.chains import TransformChain, LLMChain, SimpleSequentialChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [55]:
user_123_personal_details = PersonalDetails(first_name="",
                                last_name="",
                                full_name="",
                                phone_nr="",
                                city="",
                                email="",
                                language="")

In [56]:
user_123_personal_details

PersonalDetails(first_name='', last_name='', full_name='', phone_nr='', city='', email='', language='')

In [86]:
def ask_for_info(ask_for = ['name','age', 'location']):

    # prompt template 1
    first_prompt = ChatPromptTemplate.from_template(
        "Below is are some things to ask the user for in a coversation way. you should only ask one question at a time even if you don't get all the info \
        don't ask as a list! Don't greet the user! Don't say Hi.Explain you need to get some info. If the ask_for list is empty then thank them and ask how you can help them \n\n \
        ### ask_for list: {ask_for}"
    )
    # info_gathering_chain
    info_gathering_chain = first_prompt | structured_llm
    ai_chat = info_gathering_chain.invoke({"ask_for": ask_for})
    return ai_chat


In [87]:
ask_for_info()

PersonalDetails(first_name='John', last_name='Doe', full_name='John Doe', phone_nr='123-456-7890', city='New York', email='johndoe@example.com', language='english')

In [ ]:
text_input ="ok My name is Sam"
user_details, ask_for = filter_response(text_input, user_123_personal_details)

In [89]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI


prompt_1 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on animals."),
        ("human", "{query}"),
    ]
)
prompt_2 = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert on vegetables."),
        ("human", "{query}"),
    ]
)

chain_1 = prompt_1 | llm | StrOutputParser()
chain_2 = prompt_2 | llm | StrOutputParser()

route_system = "Route the user's query to either the animal or vegetable expert."
route_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", route_system),
        ("human", "{query}"),
    ]
)


class RouteQuery(TypedDict):
    """Route query to destination."""
    destination: Literal["animal", "vegetable"]


route_chain = (
    route_prompt
    | llm.with_structured_output(RouteQuery)
    | itemgetter("destination")
)

chain = {
    "destination": route_chain,  # "animal" or "vegetable"
    "query": lambda x: x["query"],  # pass through input query
} | RunnableLambda(
    # if animal, chain_1. otherwise, chain_2.
    lambda x: chain_1 if x["destination"] == "animal" else chain_2,
)

chain.invoke({"query": "what color are carrots"})

'Carrots are most commonly orange, but they can also be found in a variety of other colors including purple, yellow, red, and white. These different colors are due to variations in pigments such as beta-carotene (which gives the orange color), anthocyanins (which give purple and red colors), and other compounds. Each color can have slightly different nutritional properties and flavors.'